<a href="https://colab.research.google.com/github/arkeyan/Indicators/blob/main/Kompozit_RSI_up_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting TvDatafeed  openpyxl statsmodels tqdm
!pip install tradingview-screener==2.5.0


import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tqdm import tqdm
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# RSI'yi RMA (SMMA) ile hesaplama fonksiyonu
def calculate_rsi_rma(data, length=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(com=(length - 1), min_periods=length).mean()
    avg_loss = loss.ewm(com=(length - 1), min_periods=length).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Haftalık tarama yapmak için sabit zaman dilimi
def get_weekly_interval():
    return 'Haftalık', Interval.in_weekly

# BIST'teki tüm hisse senetlerini otomatik çekme
def get_all_stocks():
    # tradingview_screener'dan tüm hisseleri al
    tv = TvDatafeed()  # TVDatafeed örneği oluştur
    Hisseler = get_all_symbols(market='turkey')  # Türk hisselerini al
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]  # BIST: kısmını kaldır
    return sorted(Hisseler)  # Sıralı liste döndür

# Veriyi çekme ve RSI 50 kırılımı analizi yapma
def analyze_rsi_breakout(stock_list):
    interval_name, interval = get_weekly_interval()
    tv = TvDatafeed()

    # XU100 verisini haftalık olarak çek
    xu100_data = tv.get_hist('XU100', exchange='BIST', interval=interval, n_bars=1000)
    xu100_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

    results = []
    for stock in tqdm(stock_list, desc="Taramalar ilerliyor"):
        try:
            data = tv.get_hist(symbol=f"{stock}/XU100", exchange='BIST', interval=interval, n_bars=1000)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data['RSI'] = calculate_rsi_rma(data)

            if data['RSI'].iloc[-2] < 50 <= data['RSI'].iloc[-1]:
                results.append([stock, data['Close'].iloc[-1], "RSI 50 Üzeri", interval_name, data.index[-1]])
                print(f"{stock} - RSI 50 kırılımı - Zaman Dilimi: {interval_name}")

        except Exception as e:
            print(f"{stock} işlenirken hata: {e}")

    df_results = pd.DataFrame(results, columns=['Hisse Adı', 'Son Fiyat', 'Kırılım Türü', 'Zaman Dilimi', 'Kırılım Zamanı'])
    return df_results

# Tüm hisseleri çek
stock_list = get_all_stocks()

# Tarama yap ve sonuçları göster
df_results = analyze_rsi_breakout(stock_list)
print("RSI 50 Kırılım Sinyalleri:")
print(df_results)

# Excel dosyasına kaydet
excel_file = 'rsi_breakout_signals_weekly.xlsx'
df_results.to_excel(excel_file, index=False)
print(f"RSI 50 kırılım sinyalleri '{excel_file}' dosyasına kaydedildi.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-a47irdy9
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-a47irdy9
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=a9ff3cf3a528ad718cc7cfac7d42fbbe0cbdbec4610a6a20f6d97af4e25963ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-l1staybg/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00


Taramalar ilerliyor:   1%|          | 7/585 [00:07<10:56,  1.14s/it]

AFYON - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor:   3%|▎         | 18/585 [00:21<12:20,  1.31s/it]

AKFIS işlenirken hata: single positional indexer is out-of-bounds


...

SMART - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor:  91%|█████████▏| 534/585 [07:58<00:52,  1.03s/it]

TTKOM - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor: 100%|██████████| 585/585 [08:41<00:00,  1.12it/s]


RSI 50 Kırılım Sinyalleri:
   Hisse Adı  Son Fiyat  Kırılım Türü Zaman Dilimi      Kırılım Zamanı
0      AFYON   0.001574  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
1      ALCTL   0.013538  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
2      ALGYO   0.002175  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
3      ARDYZ   0.003869  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
4      BOBET   0.002452  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
5      BVSAN   0.010688  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
6      CRDFA   0.000725  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
7      ENJSA   0.006235  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
8      GLRYH   0.000294  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
9      GMSTR   0.021712  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
10     KUTPO   0.008070  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
11     METRO   0.000275  RSI 50 Üzeri     Haftalık 2025-01-20 06:00:00
12     OFSYM   0.004291  RSI 50 Üzeri     Haftalık